#Data wrangling& Ingestion
Collect data leveraging Twitter API and save it directly to a storage

## What exists in our file system

In [3]:
dbutils.fs.ls("/mnt/")

Out[35]: [FileInfo(path='dbfs:/mnt/<mount-name>/', name='<mount-name>/', size=0),
 FileInfo(path='dbfs:/mnt/mle2ebigdatasadata/', name='mle2ebigdatasadata/', size=0),
 FileInfo(path='dbfs:/mnt/processed/', name='processed/', size=0),
 FileInfo(path='dbfs:/mnt/raw/', name='raw/', size=0),
 FileInfo(path='dbfs:/mnt/refined/', name='refined/', size=0),
 FileInfo(path='dbfs:/mnt/refineddata/', name='refineddata/', size=0)]

# Twitter API search example

In [5]:
import twitter

consumer_key = dbutils.secrets.get(scope = "build-ml-bigdata-secrets", key = "twitter-consumer-key") #twitter app’s API Key
api_consumer_secret = dbutils.secrets.get(scope = "build-ml-bigdata-secrets", key = "twitter-api-consumer-secret") #twitter app’s API secret Key
access_token = dbutils.secrets.get(scope = "build-ml-bigdata-secrets", key = "twitter-access-token") #twitter app’s Access token
access_token_secret = dbutils.secrets.get(scope = "build-ml-bigdata-secrets", key = "twitter-access-token-secret") #twitter app’s access token secret

api = twitter.Api(consumer_key=consumer_key,
                  consumer_secret=api_consumer_secret,
                  access_token_key=access_token,
                  access_token_secret=access_token_secret)




In [6]:
# twitter api search example
search_term = 'covid-19'
since = '2020-04-27' #date in format: YYYY-MM-DD
count = '100000' # amount of tweet retrived
searchOutput = api.GetSearch(raw_query="q={}%20&result_type=recent&since={}&count={}&lang=en".format(search_term,since,count))


In [7]:
# get the object attributes:
print([s.text for s in searchOutput])
print(dir(searchOutput[0]))

['RT @CsharpCorner: Easily Read Key Vault Secrets From https://t.co/kWoL8UW9Gu Core #WebAPI #Application by @SarathlalSasee1 cc @CsharpCorner…']
['AsDict', 'AsJsonString', 'NewFromJsonDict', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_json', 'contributors', 'coordinates', 'created_at', 'created_at_in_seconds', 'current_user_retweet', 'favorite_count', 'favorited', 'full_text', 'geo', 'hashtags', 'id', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_user_id', 'lang', 'location', 'media', 'param_defaults', 'place', 'possibly_sensitive', 'quoted_status', 'quoted_status_id', 'quoted_status_id_str', 'retweet_count', 'retweeted', 'retweeted_status', 'scopes', 'source', 'text', 'truncated', 'tweet_mode', 'urls', 'user', 'user_mentions', 'withheld_copyright', 'withheld_in_countries', 'withheld_scope']

What exists in the our file system:

In [9]:
json = searchOutput[0].AsJsonString()
print(json)
df = spark.read.json(sc.parallelize([json]))
df.show(truncate=False)

{"created_at": "Wed Apr 29 13:32:13 +0000 2020", "hashtags": [{"text": "WebAPI"}, {"text": "Application"}], "id": 1255490072905949187, "id_str": "1255490072905949187", "lang": "en", "retweet_count": 2, "retweeted_status": {"created_at": "Wed Apr 29 13:30:00 +0000 2020", "favorite_count": 2, "hashtags": [{"text": "WebAPI"}, {"text": "Application"}], "id": 1255489513167581187, "id_str": "1255489513167581187", "lang": "en", "retweet_count": 2, "source": "<a href=\"https://ads-api.twitter.com\" rel=\"nofollow\">Twitter for Advertisers</a>", "text": "Easily Read Key Vault Secrets From https://t.co/kWoL8UW9Gu Core #WebAPI #Application by @SarathlalSasee1 cc\u2026 https://t.co/2PDTfr6ofX", "truncated": true, "urls": [{"expanded_url": "http://ASP.NET", "url": "https://t.co/kWoL8UW9Gu"}, {"expanded_url": "https://twitter.com/i/web/status/1255489513167581187", "url": "https://t.co/2PDTfr6ofX"}], "user": {"created_at": "Fri Jan 21 06:52:01 +0000 2011", "description": "Global community of software and data developers. Learn & Share!", "favourites_count": 6144, "followers_count": 126906, "friends_count": 6, "geo_enabled": true, "id": 241007239, "id_str": "241007239", "listed_count": 2002, "location": "Philadelphia (USA). New Delhi.", "name": "C# Corner", "profile_background_color": "0099B9", "profile_background_image_url": "http://abs.twimg.com/images/themes/theme4/bg.gif", "profile_background_image_url_https": "https://abs.twimg.com/images/themes/theme4/bg.gif", "profile_background_tile": true, "profile_banner_url": "https://pbs.twimg.com/profile_banners/241007239/1525256855", "profile_image_url": "http://pbs.twimg.com/profile_images/601645996360138752/JXX6TiZa_normal.png", "profile_image_url_https": "https://pbs.twimg.com/profile_images/601645996360138752/JXX6TiZa_normal.png", "profile_link_color": "0099B9", "profile_sidebar_border_color": "FFFFFF", "profile_sidebar_fill_color": "95E8EC", "profile_text_color": "3C3940", "profile_use_background_image": true, "screen_name": "CsharpCorner", "statuses_count": 78692, "url": "https://t.co/gx01reMdPA"}, "user_mentions": [{"id": 1033089094610341888, "id_str": "1033089094610341888", "name": "Sarathlal Saseendran", "screen_name": "SarathlalSasee1"}]}, "source": "<a href=\"http://twitter.com/download/android\" rel=\"nofollow\">Twitter for Android</a>", "text": "RT @CsharpCorner: Easily Read Key Vault Secrets From https://t.co/kWoL8UW9Gu Core #WebAPI #Application by @SarathlalSasee1 cc @CsharpCorner\u2026", "urls": [{"expanded_url": "http://ASP.NET", "url": "https://t.co/kWoL8UW9Gu"}], "user": {"created_at": "Fri Apr 11 10:15:45 +0000 2014", "description": "Nation First, Rt Not Endorsement.\n Honoured to be followed by Prime Minister @narendramodi Ji.", "favourites_count": 57901, "followers_count": 13500, "friends_count": 3151, "geo_enabled": true, "id": 2475922691, "id_str": "2475922691", "listed_count": 225, "location": "Etawah-Lucknow-Delhi-Noida", "name": "Reetesh Maheshwari", "profile_background_color": "C0DEED", "profile_background_image_url": "http://abs.twimg.com/images/themes/theme4/bg.gif", "profile_background_image_url_https": "https://abs.twimg.com/images/themes/theme4/bg.gif", "profile_banner_url": "https://pbs.twimg.com/profile_banners/2475922691/1568269350", "profile_image_url": "http://pbs.twimg.com/profile_images/1218027636993097728/3UITeJgO_normal.jpg", "profile_image_url_https": "https://pbs.twimg.com/profile_images/1218027636993097728/3UITeJgO_normal.jpg", "profile_link_color": "0084B4", "profile_sidebar_border_color": "5ED4DC", "profile_sidebar_fill_color": "95E8EC", "profile_text_color": "3C3940", "profile_use_background_image": true, "screen_name": "Reetesh777", "statuses_count": 75212}, "user_mentions": [{"id": 241007239, "id_str": "241007239", "name": "C# Corner", "screen_name": "CsharpCorner"}, {"id": 1033089094610341888, "id_str": "1033089094610341888", "name": "Sarathlal Saseendran", "screen_name": "SarathlalSasee1"}, {"id": 241007239, "id_str": "241007239", "name": "C# C

In [10]:

users = api.GetFriends()

print([u.screen_name for u in users])

['bibryam', 'mariaKhalusova', 'georgemao', 'ro14nd', 'ankinson', 'ManoMarks', 'jdegoes', 'AstasiaMyers', 'synecdotal', 'christhalinger', 'SamHeughan', 'VKazulkin', 'shaunnorris', 'rakyll', 'arnonrgo', 'GCPcloud', 'martinkl', 'DonovanBrown', 'brianlagunas', 'TheCodeTraveler', 'omrihaviv', 'ruthieyakubu', 'Adron', 'matvelloso', 'QuinnyPig', 'glaucia_lemos86', 'DrQz', 'martinwoodward', 'shankuehn', 'revodavid', 'LoriLorusso', 'hipsterelectron', 'laytoun', 'ghillert', 'boredabdel', 'baroquebobcat', 'jimgris', 'igb', 'dviqueen', 'DataDuke', 'BillyKorando', 'Sharat_Chander', 'myfear', 'steveonjava', 'TechMike2kX', 'thekayls', 'morsapaes', 'Taragrg6', 'holtbt', 'rorypreddy', 'StevenMurawski', 'shwars', 'old_sound', 'arschles', 'erikstmartin', 'AnaCidre_', 'fboucheros', 'ChloeCondon', 'DivineOps', 'szynszyliszys', 'AmandaK_Data', 'demibenari', 'rmoff', 'Westerflyer', 'frecklefacelace', 'odeded', 'andyelastacloud', 'clemensv', 'lcooney', 'codebeast', 'LareneLg', 'abhi_tweeter', 'tadmanter', 'ryguyrg', 'SjoukjeZaal', 'ThoHeller', 'rikhepworth', 'mattstratton', 'Amys_Kapers', 'StephandSec', 'devopsdom', 'luuk_mager', 'geekingthething', 'thiagoavadore', 'iamalexmang', 'ItaiYaffe', 'pjdecarlo', 'prasinous', 'MarchonAnna', 'rafik_saad', 'kubernetesio', 'ossia', 'SPCPartyPatrol', 'noopkat', 'cassidoo', 'bbenz', 'maartenballiauw', 'markjbrown', 'fincooper', 'mlevkov', 'SophieSignify', 'mipsytipsy', 'deepu105', 'iCyberFighter', 'Toni_Hasson', 'iamdevloper', 'ctrlshifti', 'TYsewyn', 'nikkimc', 'IslamAttrash', 'ashevat', 'agoncal', 'ilya_sher_prog', 'jessitron', 'livetennis', 'IrisClasson', 'alicegoldfuss', 'OlehDokuka', 'denisejacobs', 'ThomasMaurer', 'cassiecodes', 'amitaistern', 'g33konaut', 'psybercity', 'fokslukasz', 'crazy4pi314', 'simonwaight', 'mp911de', 'sallybrowner76', 'mai_isthebest', 'minxdragon', 'AnnoymousGiraf', 'juliendubois', 'Howell_MSFT', 'gwencitaaa', 'erikzaadi', 'hhariri', 'manekinekko', 'jenlooper', 'MatanYungman', 'AdiShavit', 'webmaxru', 'sabinajohansso5', 'magrom', 'tibor19', 'GlennColpaert', 'noopman', 'scottcoulton', 'alexoeducative', 'DynamicWebPaige', 'MSFTReactor', 'guywiener', 'IAmOnDemand', 'jonahbalfour', 'arkodyutisaha', 'YRodenski', 'BigDataBorat', 'Duncanma', 'ThePracticalDev', 'turaaaa', 'JavaAfrica', 'brian_armstrong', 'sethjuarez', 'raulraja', 'NatalyCreates', 'sec_tigger', 'daisychaussee', 'MaximRouiller', 'yashar', 'holdenkarau', 'Rick345', 'unicodeveloper', 'CYarkoni', 'davidsancheza', 'itaysk', 'jameswoollard', 'trikkser', 'IanMmmm', 'shar1z', 'jaydestro', 'RapoportAnat', 'katierobbert', 'ixek', 'John_Papa', 'magwanjiru', 'RainNoa', 'DaveKerpen', 'imcdnzl', 'SaschaDittmann', 'MWatsonSYD1', 'developerjack', 'tirsen', 'jonnyzzz', 'azureadvocates', 'SwissCognitive', 'RwandaRob', 'romiem', 'kellyshalk', 'lee_stott', 'SarahDavis816', 'scottgu', 'ido_co', 'nebrius', 'ScottCate', 'ASpittel', 'tenderlove', 'sxywu', 'SternTwena', 'adamj89', 'aprilwensel', 'hmason', 'Ronald_vanLoon', 'schmarzo', 'DrDataScientist', 'ploeh', 'databricks', 'sherrrylst', 'cmaneu', 'MrAhmadAwais', 'Cassieview', 'frlazzeri', 'AysSomething', 'codinghorror', 'nitya', 'rinaarts', 'mootpointer', 'LBugnion', 'MoonPie', 'proudboffin', 'markrussinovich', 'nthonyChu', 'chadfowler', 'bitandbang', 'spboyer', 'as_w', 'truckerfling', 'MichaelBender', 'dana_baril', 'buffer', 'yaelimpressions', 'wiredcanuck', 'jeremylikness', 'ElissaBeth', 'b0rk', 'paladique', 'ayende', 'martinfowler', 'UdiDahan', 'dcominottim', 'lcalcote', 'Brian_Peel', 'emilylrich', 'ryan_levick', 'chris_noring', 'MikeCodesDotNET', 'deanbryen', 'notriky_', 'CollinsKKorir', 'linhdaosmooke', 'film_girl', 'annie_parker', 'DavidSmooke', 'jessfraz', 'sarah_edo', 'CoolAssPuppy', 'jaceklaskowski', 'KotlinWeekly', 'Azure', 'yoavmatchulsky', 'intellijidea', 'trisha_gee', 'chanezon', 'sjmaple', 'mkheck', 'starbuxman', 'kennybastani', 'hackernoon', 'VisualStudio', 'TheRegister', 'Fox0x01', 'yoshioterada', 'jawache', 'julietlovesjava', 'slace', 'relizarov', 'ShmuelaJ', 'Auguronomics', 'levy

Work with blob service

In [12]:
# create a blob service
blob_service = BlockBlobService(
    account_name=azure_storage_account_name,
    sas_token=azure_storage_sas_token,
)


# download metadata.csv
metadata_filename = 'metadata.csv'
metadata = blob_service.get_blob_to_path(
    container_name=container_name,
    blob_name=metadata_filename,
    file_path=metadata_filename
)


In [13]:
print(dir(blob_service))

['MAX_BLOCK_SIZE', 'MAX_CHUNK_GET_SIZE', 'MAX_SINGLE_GET_SIZE', 'MAX_SINGLE_PUT_SIZE', 'MIN_LARGE_BLOCK_UPLOAD_THRESHOLD', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__metaclass__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply_host', '_copy_blob', '_get_blob', '_get_host_locations', '_httpclient', '_lease_blob_impl', '_lease_container_impl', '_list_blobs', '_list_containers', '_perform_request', '_put_blob', '_put_block', '_put_block_list', 'abort_copy_blob', 'account_key', 'account_name', 'acquire_blob_lease', 'acquire_container_lease', 'authentication', 'blob_type', 'break_blob_lease', 'break_container_lease', 'change_blob_lease', 'change_container_lease', 'copy_blob', 'create_blob_from_bytes', 'create_blob_from_path', 'create_blob_from_stream', 'create_blob_from_text', 'create_container', 'delete_blob', 'delete_container', 'exists', 'generate_account_shared_access_signature', 'generate_blob_shared_access_signature', 'generate_container_shared_access_signature', 'get_blob_metadata', 'get_blob_properties', 'get_blob_service_properties', 'get_blob_service_stats', 'get_blob_to_bytes', 'get_blob_to_path', 'get_blob_to_stream', 'get_blob_to_text', 'get_block_list', 'get_container_acl', 'get_container_metadata', 'get_container_properties', 'is_emulated', 'key_encryption_key', 'key_resolver_function', 'list_blobs', 'list_containers', 'location_mode', 'make_blob_url', 'primary_endpoint', 'protocol', 'put_block', 'put_block_list', 'release_blob_lease', 'release_container_lease', 'renew_blob_lease', 'renew_container_lease', 'request_callback', 'request_session', 'require_encryption', 'response_callback', 'retry', 'retry_callback', 'sas_token', 'secondary_endpoint', 'set_blob_metadata', 'set_blob_properties', 'set_blob_service_properties', 'set_container_acl', 'set_container_metadata', 'set_proxy', 'set_standard_blob_tier', 'snapshot_blob', 'socket_timeout']

In [14]:
container_name = "tweets-covid19"
blobs = blob_service.list_blobs(container_name)
sorted_blobs = sorted(list(blobs), key=lambda e: e.name, reverse=True)

In [15]:
import re
dirs = {}

pattern = '([\w]+)\/([\w]+)\/([\w.]+).json'
for blob in sorted_blobs:
    
    m = re.match(pattern, blob.name)
    
    if m:
        dir_ = m[1] + '/' + m[2]
        
        if dir_ in dirs:
            dirs[dir_] += 1
        else:
            dirs[dir_] = 1
        
dirs

<>:4: DeprecationWarning: invalid escape sequence \w
<>:4: DeprecationWarning: invalid escape sequence \w
<command-4454164535667233>:4: DeprecationWarning: invalid escape sequence \w
 pattern = '([\w]+)\/([\w]+)\/([\w.]+).json'
Out[15]: {'noncomm_use_subset/pmc_json': 2241,
 'noncomm_use_subset/pdf_json': 2526,
 'custom_license/pmc_json': 8036,
 'custom_license/pdf_json': 27626,
 'comm_use_subset/pmc_json': 9260,
 'comm_use_subset/pdf_json': 9637,
 'biorxiv_medrxiv/pdf_json': 2121}

In [16]:
blob_service.copy_blob(container_name, blob_name, copy_source, metadata=None, source_if_modified_since=None, source_if_unmodified_since=None, source_if_match=None, source_if_none_match=None, destination_if_modified_since=None, destination_if_unmodified_since=None, destination_if_match=None, destination_if_none_match=None, destination_lease_id=None, source_lease_id=None, timeout=None)